In [1]:
import torch
import numpy as np
from idem_net_mnist import IdemNetMnist
from idem_net_celeba import IdemNetCeleba
from data_loader import load_MNIST
import matplotlib.pyplot as plt

/Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/je/anaconda3/envs/DL-Idempotant/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/je/anaconda3/envs/DL-Idempotant/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Othe

In [4]:
run_id = "celeba20241113-154812"
epoch_num = "_final.pth"

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')


checkpoint_path = f"checkpoints/{run_id}/{epoch_num}"
device

device(type='mps')

In [10]:
model = IdemNetCeleba(3)

state_dict = torch.load(checkpoint_path, weights_only=True, map_location=device)
#state_dict = state_dict["model_state_dict"]

In [11]:
model.load_state_dict(state_dict)
#model.eval()

<All keys matched successfully>

In [16]:
def plot_generation(inputs, outputs, outputs2, num_images=5, show=False):
    """
    Plots input and output image pairs side by side.
    
    Parameters:
    - inputs: Batch of input images, expected shape (batch_size, height, width).
    - outputs: Batch of output images, expected shape (num_applications, batch_size, height, width).
    - num_images: Number of image pairs to display (default is 5).
    """
    # Limit the number of images to the smaller of num_images or batch size
    num_images = min(num_images, len(inputs), len(outputs))
    
    plt.figure(figsize=(8, num_images * 3))
    for i in range(num_images):
        # Plot input image
        plt.subplot(num_images, 3, 2 * i + 1)
        plt.imshow(inputs[i].squeeze().permute(1,2,0).detach().numpy())
        plt.title("Input")
        plt.axis('off')
        
        # Plot output image
        plt.subplot(num_images, 3, 2 * i + 2)
        plt.imshow(outputs[i].squeeze().permute(1,2,0).detach().numpy())
        plt.title("Output")
        plt.axis('off')

        # Plot output image
        plt.subplot(num_images, 3, 2 * i + 3)
        plt.imshow(outputs2[i].squeeze().permute(1,2,0).detach().numpy())
        plt.title("Output")
        plt.axis('off')

    plt.tight_layout()
    if show:
        plt.show()



In [18]:
# Generate random images based on noise with the same shape as the input images
num_noise = 2
noise = torch.randn(num_noise, 3, 64, 64)

# Do interpolation between the first two images
num_interpolations = 30
interpolations = torch.zeros(num_interpolations, 1, 3, 64, 64)
outputs = torch.zeros(num_interpolations, 1, 3, 64, 64)
outputs2 = torch.zeros(num_interpolations, 1, 3, 64, 64)
for i in range(num_interpolations):
    alpha = i / (num_interpolations - 1)
    interpolations[i] = alpha * noise[0] + (1 - alpha) * noise[1]





In [19]:
import os

output_dir = "output_images_latentspace_celeba"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# output for interpolations
for i in range(num_interpolations):
    outputs[i] = model(interpolations[i])
    outputs2[i] = model(outputs[i])
    plot_generation(interpolations[i], outputs[i], outputs2[i], num_images=1)
    # Save plot
    plt.savefig(os.path.join(output_dir, f"interpolation_celeba_{i}.png"))
    plt.close()


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-4.0498314..4.0967436].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.014082886..0.9907379].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0071496265..0.98580426].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-3.9136791..3.9603755].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.014281568..0.99082726].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-0.0063443747..0.98597515].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got ran